In [1]:
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging
import xarray as xr
import os

In [28]:
folder = "./Visibility csv/"

In [29]:
file_paths = os.listdir(folder)
data = [pd.read_csv(folder + path) for path in file_paths]

In [30]:
data[0]

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,SOURCE,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,VIS
0,40292099999,"ER RABBAH, JO",31.266667,35.7500,920.0,2024-09-28T09:00:00,4,FM-12,99999,V020,"010000,1,9,9"
1,40292099999,"ER RABBAH, JO",31.266667,35.7500,920.0,2024-12-22T06:00:00,4,FM-12,99999,V020,"010000,1,9,9"
2,40305099999,"JAFER, JO",30.283333,36.1500,853.0,2020-12-10T03:00:00,4,FM-12,99999,V020,"010000,1,9,9"
3,40305099999,"JAFER, JO",30.283333,36.1500,853.0,2020-12-10T06:00:00,4,FM-12,99999,V020,"010000,1,9,9"
4,40305099999,"JAFER, JO",30.283333,36.1500,853.0,2020-12-10T09:00:00,4,FM-12,99999,V020,"010000,1,9,9"
...,...,...,...,...,...,...,...,...,...,...,...
210680,40192199999,"SEDOM, IS",31.030600,35.3919,-388.0,2025-01-20T19:00:00,4,FM-12,99999,V020,"999999,9,9,9"
210681,40192199999,"SEDOM, IS",31.030600,35.3919,-388.0,2025-01-20T20:00:00,4,FM-12,99999,V020,"999999,9,9,9"
210682,40192199999,"SEDOM, IS",31.030600,35.3919,-388.0,2025-01-20T21:00:00,4,FM-12,99999,V020,"999999,9,9,9"
210683,40192199999,"SEDOM, IS",31.030600,35.3919,-388.0,2025-01-20T22:00:00,4,FM-12,99999,V020,"999999,9,9,9"


now for column "VIS", i want to take the number before the comma ",".

In [31]:
data[0]['VIS'] = data[0]['VIS'].str.split(',', n=1).str[0].astype(float)

In [32]:
data[0]['NAME'].unique()

array(['ER RABBAH, JO', 'JAFER, JO', 'QUEEN ALIA INTERNATIONAL, JO',
       'WADI RAYAN, JO', 'GHOR SAFI, JO', 'MARKA INTERNATIONAL, JO',
       'PRINCE HASAN, JO', 'RAS MUNEEF, JO', 'DEIR ALLA, JO',
       'QATRANEH, JO', 'AQABA KING HUSSEIN INTERNATIONAL, JO',
       'IRBID MET, JO', 'KING HUSSEIN, JO', 'MA AN, JO', 'H4, JO',
       'SEDOM, IS'], dtype=object)

In [33]:
data2 = []
for name in data[0]['NAME'].unique():
    df_name = data[0][data[0]['NAME'] == name]
    data2.append(df_name)

In [34]:
data = data2

In [35]:
for i in range(len(data)):
    data[i]['DATE'] = pd.DatetimeIndex(data[i]['DATE'])
    data[i].loc[data[i]['VIS'] == 999999, 'VIS'] = np.nan
    data[i].loc[data[i]['VIS'] == 0.0, 'VIS'] = np.nan

C:\Users\user\AppData\Local\Temp\ipykernel_5908\2059098168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i]['DATE'] = pd.DatetimeIndex(data[i]['DATE'])


In [37]:
grouped = []
date = pd.DataFrame()
date['DATE'] = pd.date_range('2020-12-10', '2025-01-20')
date.set_index('DATE', inplace=True)
for i in range(len(data)):
    grouped.append(data[i].groupby(data[i]['DATE'].dt.date)[['LATITUDE','LONGITUDE','ELEVATION','VIS']].mean())
    grouped[i].set_index(pd.DatetimeIndex(grouped[i].index), inplace=True)
    grouped[i] = pd.concat([grouped[i], date],axis=1)
    grouped[i]['LONGITUDE'] = grouped[i]['LONGITUDE'][0]
    grouped[i]['LATITUDE'] = grouped[i]['LATITUDE'][0]
    grouped[i]['ELEVATION'] = grouped[i]['ELEVATION'][0]
    grouped[i] = grouped[i][grouped[i].index >= '10-12-2020']
    grouped[i].reset_index(inplace=True)

In [53]:
grouped[2]

,DATE,LATITUDE,LONGITUDE,ELEVATION,VIS
0,2020-12-10,31.722556,35.993214,729.99,5212.121212
1,2020-12-11,31.722556,35.993214,729.99,5531.250000
2,2020-12-12,31.722556,35.993214,729.99,5181.250000
3,2020-12-13,31.722556,35.993214,729.99,5322.580645
4,2020-12-14,31.722556,35.993214,729.99,4277.777778
...,...,...,...,...,...
1498,2025-01-16,31.722556,35.993214,729.99,4750.000000
1499,2025-01-17,31.722556,35.993214,729.99,4875.000000
1500,2025-01-18,31.722556,35.993214,729.99,4687.500000
1501,2025-01-19,31.722556,35.993214,729.99,4578.125000


In [55]:
df = pd.concat(grouped).drop(columns=['ELEVATION'])

In [66]:
days = pd.date_range('2020-12-10', '2025-01-20')
grid_lon = np.round(np.arange(35.5, 37.41, 0.01),2)
grid_lat = np.round(np.arange(32.2, 31.0, -0.01),2)
k = np.empty((len(days),len(grid_lat) , len(grid_lon)))
k[:] = np.nan

for i in range(len(days)):
    sdf = df[df['DATE'] == days[i]]
    if len(sdf.dropna()) > 2:
        sdf = sdf.dropna()
        lon = sdf['LONGITUDE'].values
        lat = sdf['LATITUDE'].values

        OK = OrdinaryKriging(lon, lat, sdf['VIS'].values, variogram_model='linear')
        k[i], ss = OK.execute('grid', grid_lon, grid_lat)

In [67]:
dat = xr.Dataset({'VIS':(("time","latitude" ,"longitude"), k)},
                 coords={
        "time": days,
        "latitude": grid_lat,
        "longitude": grid_lon,
        
    })

In [68]:
dat

<xarray.Dataset>
Dimensions:    (time: 1503, latitude: 121, longitude: 191)
Coordinates:
  * time       (time) datetime64[ns] 2020-12-10 2020-12-11 ... 2025-01-20
  * latitude   (latitude) float64 32.2 32.19 32.18 32.17 ... 31.02 31.01 31.0
  * longitude  (longitude) float64 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
Data variables:
    VIS        (time, latitude, longitude) float64 8.899e+03 ... 7.522e+03

In [72]:
# Number of chunks
n = 4

# Get indices to split along the time dimension
time_len = dat.dims["time"]
splits = np.array_split(np.arange(time_len), n)

# Save each chunk to a separate file
for i, idx in enumerate(splits):
    ds_subset = dat.isel(time=idx)
    ds_subset.to_netcdf(f"./Visibility nc/Visibility_{i+1}.nc")